In [2]:
using Interact
using Mux
using Plots
using DelimitedFiles
using Interpolations

┌ Info: Precompiling Mux [a975b10e-0019-58db-a62f-e48ff68538c9]
└ @ Base loading.jl:1278
┌ Info: Precompiling Plots [91a5bcdd-55d7-5caf-9e0b-520d859cae80]
└ @ Base loading.jl:1278
┌ Info: Precompiling Interpolations [a98d9a8b-a2ab-59e6-89dd-64a1c18fca59]
└ @ Base loading.jl:1278


### read files

In [3]:
Ns=1:10
νs=(1/60)*(0.2:0.2:10.0)
r=readdlm("data/r_squared_delay.txt")
s=readdlm("data/s_squared_delay.txt")
t=readdlm("data/t_squared_delay.txt");

In [4]:
for i in 1:size(r)[1], j in 1:size(r)[2] 
    r[i,j]=min(r[i,j],1)
end

In [5]:
max_r=zeros(length(Ns))
i=0
for N in Ns
    i+=1
    max_r[i]=60*νs[findfirst(x->x>0.05,r[i,:])]
end
colors=[:green, :blue, :purple, :red];

## interpolations and scaling plots

In [ ]:
data=s
sitp = scale(interpolate(data, BSpline(Cubic(Line(OnGrid())))), Ns, νs)

@manipulate for top in 0.2:0.1:10.0
xmin,xmax,ymin,ymax=(0.2,top,1,10)
pmin=findfirst(p->(xmax*p>xmin && ymax*p>ymin),0:0.01:1)
p=(0.01*pmin):0.01:1

X,Y=(xmax*p, ymax*p)
Z=[sitp(Y[k],X[k]./60) for k in 1:length(X)]
plot2=plot(p,Z,xlim=[0,1],ylim=[0.58,1.4],xlab="along straight line")

X=(0.2:0.05:10.0)./60
Y=1.0:0.05:10.0
S=[sitp(y,x) for y in Y, x in X]
plot1=heatmap(60*X,Y,S,xlab="requests per minute",ylab="number of buses",colorbar_title="driven distance",legend=:bottomright,
    yticks=Ns,aspectratio=.7,c=colors,clim=(0.7,1.7),frame=:box,size=[800,500],left_margin=8Plots.mm)
plot!(max_r,Ns,linewidth=2,linecolor=:orange,markersize=5,markercolor=:orange,marker=:circle,label=">5% rejected")
plot!(xmax*p,ymax*p,linewidth=3,linecolor=:silver,label="straight line")

plot(plot1,plot2,size=[1200,400])
end

┌ Info: Precompiling GR_jll [d2c73de3-f751-5644-a686-071e5b155ba9]
└ @ Base loading.jl:1278


In [ ]:
data=t
sitp = scale(interpolate(data, BSpline(Cubic(Line(OnGrid())))), Ns, νs)

@manipulate for top in 0.2:0.1:10.0
xmin,xmax,ymin,ymax=(0.2,top,1,10)
pmin=findfirst(p->(xmax*p>xmin && ymax*p>ymin),0:0.01:1)
p=(0.01*pmin):0.01:1

X,Y=(xmax*p, ymax*p)
Z=[sitp(Y[k],X[k]./60) for k in 1:length(X)]
plot2=plot(p,Z,xlim=[0,1],ylim=[0.5,5],xlab="along straight line")

X=(0.2:0.05:10.0)./60
Y=1.0:0.05:10.0
S=[sitp(y,x) for y in Y, x in X]
plot1=heatmap(60*X,Y,S,xlab="requests per minute",ylab="number of buses",colorbar_title="delay time",legend=:bottomright,
    yticks=Ns,aspectratio=.7,c=colors,clim=(0.7,5),frame=:box,size=[800,500],left_margin=8Plots.mm)
plot!(max_r,Ns,linewidth=2,linecolor=:orange,markersize=5,markercolor=:orange,marker=:circle,label=">5% rejected")
plot!(xmax*p,ymax*p,linewidth=3,linecolor=:silver,label="straight line")

plot(plot1,plot2,size=[1200,400])
end

## app

In [ ]:
data_t=t
sitp_t = scale(interpolate(data_t, BSpline(Cubic(Line(OnGrid())))), Ns, νs)

data_s=s
sitp_s = scale(interpolate(data_s, BSpline(Cubic(Line(OnGrid())))), Ns, νs)

ui=@manipulate for top in 0.2:0.1:10.0
xmin,xmax,ymin,ymax=(0.2,top,1,10)
pmin=findfirst(p->(xmax*p>xmin && ymax*p>ymin),0:0.01:1)
p=(0.01*pmin):0.01:1

    #t
X,Y=(xmax*p, ymax*p)
Z=[sitp_t(Y[k],X[k]./60) for k in 1:length(X)]
plot2=plot(p,Z,xlim=[0,1],ylim=[0.7,8],xlab="along straight line",ylab="delay time",frame=:box,legend=:none)

X=(0.2:0.05:10.0)./60
Y=1.0:0.05:10.0
S=[sitp_t(y,x) for y in Y, x in X]
plot1=heatmap(60*X,Y,S,xlab="requests per minute",ylab="number of buses",title="delay time",legend=:bottomright,
    yticks=Ns,aspectratio=.7,c=colors,clim=(0.7,5),frame=:box,size=[800,500],left_margin=8Plots.mm)
plot!(max_r,Ns,linewidth=2,linecolor=:orange,markersize=5,markercolor=:orange,marker=:circle,label=">5% rejected")
plot!(xmax*p,ymax*p,linewidth=3,linecolor=:silver,label="straight line")
    
    #s
X,Y=(xmax*p, ymax*p)
Z=[sitp_s(Y[k],X[k]./60) for k in 1:length(X)]
plot4=plot(p,Z,xlim=[0,1],ylim=[0.5,1.7],xlab="along straight line",ylab="driven distance",frame=:box,legend=:none)

X=(0.2:0.05:10.0)./60
Y=1.0:0.05:10.0
S=[sitp_s(y,x) for y in Y, x in X]
plot3=heatmap(60*X,Y,S,xlab="requests per minute",ylab="number of buses",title="driven distance",legend=:bottomright,
    yticks=Ns,aspectratio=.7,c=colors,clim=(0.5,1.7),frame=:box,size=[800,500],left_margin=8Plots.mm)
plot!(max_r,Ns,linewidth=2,linecolor=:orange,markersize=5,markercolor=:orange,marker=:circle,label=">5% rejected")
plot!(xmax*p,ymax*p,linewidth=3,linecolor=:silver,label="straight line")
    
plot(plot1,plot2,plot3,plot4,size=[1000,700])
end;

In [ ]:
@layout! ui vbox(hbox(:top), observe(_));

In [ ]:
WebIO.webio_serve(page("/", req -> ui), 8102);